In [ ]:
import numpy as np
import cv2
import os
import glob
import json
import operator


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    x1 = max(boxA[0], boxB[0])
    y1 = max(boxA[1], boxB[1])
    x2 = min(boxA[2], boxB[2])
    y2 = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    # compute the area of both the prediction and ground-truth rectangles
    boxA_area = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxB_area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(boxA_area + boxB_area - intersection_area)

    # return the intersection over union value
    return iou

In [ ]:
import operator
with open("/content/drive/MyDrive/PFE_Master/video_prediction/Real_Data_Validation/predictions.json") as file:
    data = json.load(file)

# Create a dictionary to store the distinct objects with the maximum score
filtered_data = {}
image_width = 640
image_height = 360
for item in data:
    image_id = item['image_id']
    score = item['score']

    if image_id not in filtered_data or score > filtered_data[image_id]['score']:
        filtered_data[image_id] = item

# Convert the dictionary values to a list
result = list(filtered_data.values())

for item in result:
  item['bbox'][0]=item['bbox'][0]/image_width
  item['bbox'][2]=item['bbox'][2]/image_width
  item['bbox'][1]=item['bbox'][1]/image_height
  item['bbox'][3]=item['bbox'][3]/image_height


# export the new point of bounding box in new json file
with open('/content/drive/MyDrive/PFE_Master/video_prediction/Real_Data_Validation/ouput_predicitions.json', 'w') as file:
  json.dump(result, file)

In [ ]:
import json
import operator
# read résult of prédection
with open('/content/drive/MyDrive/PFE_Master/video_prediction/Real_Data_Validation/ouput_predicitions.json') as file:
    data = json.load(file)
# read only the labeling of validation data
real_labeling_path="/content/drive/MyDrive/PFE_Master/Project/YOLOV8FireDetection_V1/datasets/Real_validation"
file_list= glob.glob(real_labeling_path+ "/*.txt")
iou=0
for file_name in file_list:
  # extract the point of bounding box predected
  name=file_name.split('/')[-1].split('.')[0]
  object_predict= list(filter(lambda x: x["image_id"] == name, data))
  if len(object_predict) !=0 :
    boxA= [object_predict[0]["bbox"][0],object_predict[0]["bbox"][1],object_predict[0]["bbox"][2],object_predict[0]["bbox"][3]]
    #extract the point of real bounding box
    file_real_path= os.path.join(real_labeling_path, file_name)
    with open(file_real_path, 'r') as file:
      content = file.readline()
      class_index, x1, y1, x2, y2 = map(float, content.strip().split())
      boxB= [x1, y1, x2, y2]
      # get % of intersection over union of my predection data (IOU)
      if intersection_over_union(boxA, boxB) >= 0.5:
        iou= iou + 1
      #print("IOU =>   ", intersection_over_union(boxA, boxB))
print("Intersection Over Union of Real data =>  ",iou/len(data))



Intersection Over Union of Real data =>   0.5443458980044346
